### 데이터 받아오기

In [35]:
import json
import numpy as np
import random
import nltk
from random import random
from random import randrange
from sklearn import datasets

In [36]:
with open("reviews_Books_5.json","r") as data_file:
    raw_data=data_file.readlines()

In [37]:
json_list = []
for i in range(len(raw_data)):
    try:
        json_data = json.loads(raw_data[i])
        json_list.append(json_data)
    except:
        print(raw_data[i],i)

{"reviewerID": "AAXUNK0W2DZG5", "asin": "0060520841", "reviewerName": "Amazon Customer \"leneker\"", "helpful": [5, 10], "reviewText": "1996 Bernard Goldberg wrote an editorial for the Wall Street Journal that said there was an obvious bias on the part of network new shows  for the liberal point of view.  he then illustrated this with an example that he dissected in-depth.  The reaction to this observation was the ruination of her career, and the beginning of his status as a pariah among most newsmen.  This book is used to add more ammo to the controversy.That the journalists who so eagerly pry into other peoples lives and business should be reluctant to be examined is hardly surprising.  Almost no one really wants to have his life or business dissected by Mike Wallace not even Dan Rather.  Some facts in this book are really potent such as the survey results which show how the average journalist and the average American are often vastly at odds.  Other chapters highlight different stor

### 데이터 분리하기

In [38]:
review_data = []
for i in range(len(json_list)):
    if (len(json_list[i]["reviewText"])!=0):
        review_dict={}
        review_dict["reviewText"] = json_list[i]["reviewText"]
        review_dict["overall"] = json_list[i]["overall"]
        review_data.append(review_dict)

In [39]:
review_data_1=[]
for i in range(len(review_data)):
    if (review_data[i]["overall"]!=4.0 and review_data[i]["overall"]!=5.0):
        review_data_1.append(review_data[i])

In [40]:
from random import random
from random import randrange
def subsample(dataset, ratio=2.0):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

In [41]:
overall_1= subsample(review_data_1)

In [42]:
reviewText = []
for i in range(len(review_data)):
    if (review_data[i]["overall"]==4.0 or review_data[i]["overall"]==5.0): reviewText.append(review_data[i]["reviewText"])

In [43]:
for i in range(len(overall_1)):
    reviewText.append(overall_1[i]['reviewText'])

In [44]:
y_data = []
for i in range(len(review_data)):
    if (review_data[i]["overall"]==4.0 or review_data[i]["overall"]==5.0): y_data.append(review_data[i]["overall"])

In [45]:
for i in range(len(overall_1)):
    y_data.append(overall_1[i]["overall"])

In [46]:
from collections import Counter
Counter(y_data)

Counter({1.0: 8499, 2.0: 10297, 3.0: 20238, 4.0: 21555, 5.0: 52105})

<h3>데이터 전처리</h3>

In [47]:
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re

In [48]:
def review_to_words(raw_review):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    
   
    cleaned_text_list=[]
    for i in range(len(raw_review)):
        
        review_text = BeautifulSoup(raw_review[i],"html.parser").get_text()
        letters_only = re.sub("[^a-zA-Z]", " ", review_text)
        words = letters_only.lower().split()
        cleaned_text = " ".join(words)
        cleaned_text_list.append(cleaned_text)

    return cleaned_text_list

       

In [49]:
pre_data=review_to_words(reviewText)

### 가장 작은 평점으로 갯수 맞추기

In [50]:
from collections import Counter
length=sorted(Counter(y_data).most_common())[0][1]

In [51]:
search=[1.0,2.0,3.0,4.0,5.0]
count=0
text=[]
y=[]

for i in search:
    count=0
    for index,value in enumerate(y_data):
        if (count==length): break
        elif (i==value):
            text.append(pre_data[index])
            y.append(value)
            count=count+1

In [52]:
Counter(y)

Counter({1.0: 8499, 2.0: 8499, 3.0: 8499, 4.0: 8499, 5.0: 8499})

<h3>데이터 셔플 다시</h3>

In [53]:
import random
c = list(zip(text, y))
random.shuffle(c)

In [54]:
a, b = zip(*c)

In [55]:
reviewText=list(a)
overall=np.array(list(b))

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
#vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True,lowercase=True)
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(reviewText)

In [57]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tfidf , overall , test_size=0.33, random_state=42)

In [58]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((28471, 1220759), (28471,), (14024, 1220759), (14024,))

### 예측

### LogisticRegression

In [59]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression(C=2.0,random_state=42,solver='sag',multi_class='multinomial',warm_start=True)
logreg.fit(x_train, y_train)

LogisticRegression(C=2.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=42, solver='sag',
          tol=0.0001, verbose=0, warm_start=True)

In [60]:
from sklearn.metrics import accuracy_score
accuracy_score(logreg.predict(x_test), y_test)

0.74693382772390193

In [61]:
from sklearn.metrics import precision_score
precision_score(logreg.predict(x_test), y_test,average=None) 

array([ 0.88569425,  0.78977891,  0.68439716,  0.60719424,  0.76501955])

In [62]:
from sklearn.metrics import classification_report
print(classification_report(y_test, logreg.predict(x_test)))

             precision    recall  f1-score   support

        1.0       0.84      0.89      0.86      2852
        2.0       0.79      0.79      0.79      2759
        3.0       0.72      0.68      0.70      2820
        4.0       0.65      0.61      0.63      2780
        5.0       0.73      0.77      0.75      2813

avg / total       0.74      0.75      0.75     14024



### INPUT 

In [63]:
texts = ["""I enjoyed this book, as long as I kept in mind that the book was written for young adults.  
         The characters were believable and it was easy to identify with their pain.""",
        "I think the author gets lost in his own thoughts and trys to make storys of something that's not there.",
        """I was very excited to start reading this book and although I did like it, 
        I thought that a lot of the words to describe body parts was kind of high-schoolish.  
        Didn't care for the cliff hanging ending.""",
        """I don't know if you are familiar with FANFIC on the internet but that what this book reminded me of.  
        I love the series and thought I would like the books...not so much.""",
        """As I love western era romances, this one was wonderful.  An easy read and ended the way we always hope....happily!"""]
overall=[4,1,3,2,5]
vecs = vectorizer.transform(review_to_words(texts)) 

In [64]:
print(overall,logreg.predict(vecs))

[4, 1, 3, 2, 5] [ 4.  1.  3.  3.  5.]
